# 🤖 Modelado y Evaluación del Proyecto: Clasificación del Estado Laboral

Este notebook corresponde a la tercera entrega del proyecto de Aprendizaje Automático. En él se entrena y evalúa un modelo de clasificación sobre datos de la Encuesta Permanente de Hogares (EPH) del INDEC para predecir la condición laboral.

In [ ]:
# 📦 Carga de librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

## 📁 Carga del dataset

In [ ]:
# Cargar el dataset limpio desde data/processed/
df = pd.read_csv('../data/processed/EPH_T123_filtrado_procesado.csv')
df.head()

## 🔄 Preprocesamiento final

In [ ]:
# Separar X e y
X = df.drop(columns='target')
y = df['target']

# Codificar variables categóricas
X = pd.get_dummies(X, drop_first=True)

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 🔍 Entrenamiento: Regresión Logística

In [ ]:
# Entrenar modelo
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)

## 📊 Evaluación del modelo

In [ ]:
# Predicciones
y_pred = model.predict(X_test)

# Métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Desocupado', 'Ocupado'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión')
plt.show()

## 📌 Conclusiones

- El modelo de regresión logística permite predecir la condición laboral con métricas aceptables considerando el desbalance.
- Las variables sociodemográficas como edad, sexo y nivel educativo muestran relación con el estado laboral.
- Se pueden evaluar otros modelos en el futuro (árboles, random forest) para comparar desempeño.